In [ ]:
import os
import pandas as pd
import pulp as pl

os.chdir("./../")
from src.VariableNameConversion import VariableNameConversion

print(os.getcwd())


In [ ]:
requested_number_of_farms = 1000
USE_CASE = "greece"
BASE_PATH = f"./data/use_case_{USE_CASE}"
YEAR = "2014"

TOTALS_VARIABLES = ["cultivatedArea"]

vnc = VariableNameConversion(BASE_PATH, USE_CASE, YEAR, TOTALS_VARIABLES)

sp_end, _, _ = vnc.main()

In [8]:
data = pd.read_csv("./../microdata_for_cost_crop_computing.csv")

data[["specificCropCosts"]].head()
data = sp_end.copy(deep=True)

In [ ]:
data[[c for c in data.columns if c.startswith("CER")]]

In [23]:
class CostCropComputer():
    def __init__(self, data):
        
        #farms = random.sample(range(data.shape[0]), 200)
        #data = data.iloc[farms]
        
        # Process data
        self.data, self.A, self.y, self.crops = self._preprocess_data(data)
        
        self.farms = self.data.shape[0]


    def _preprocess_data(self, data):
        """
        Select required variables to compute crop specific costs
        """
        
        cultivatedArea_variables = [var for var in data.columns if ".cultivatedArea" in var]
        quantitySold_variables = [var for var in data.columns if ".quantitySold" in var]
        crops = [var.replace(".cultivatedArea", "") for var in cultivatedArea_variables]

        data = data[cultivatedArea_variables + quantitySold_variables + ["specificCropCosts"]]

        data["totalCultivatedArea"] = data.apply(lambda x: x[cultivatedArea_variables].sum(), axis=1)

        data["totalCost"] = data.apply(lambda x: x["specificCropCosts"]*x["totalCultivatedArea"], axis=1)

        data = data[(data["totalCultivatedArea"]>0)&(data["specificCropCosts"]>0)]
        
        A = data[quantitySold_variables].to_numpy()
        y = data["totalCost"].to_numpy()

        return data, A, y, crops
    

    def main(self):

        model = pl.LpProblem(name="crop-cost-computing", sense=pl.LpMinimize)

        # Initialize decison variables | costs per crop
        # x = [x_{crop1}, x_{crop2}, ..., x_{cropm}]
        x_crop = {}

        for crop in self.crops:
            x_crop[crop] = pl.LpVariable(name=f"x_{crop}", lowBound=0, upBound=100)

        # Initialize slack variables
        h_slack = {}
        for farm in range(self.farms):
            h_slack[farm] = pl.LpVariable(name=f"h_{farm}", lowBound=0)
        
        # Define equations
        for i, farm in enumerate(range(self.farms)):
            equation = self.y[i] - h_slack[i] - pl.lpSum([(x_crop[crop]*self.A[i, j]) for j, crop in enumerate(self.crops)]) == 0
            model += equation

        # Define objective function
        model += pl.lpSum(list(h_slack.values()))
        
        
        #model.writeLP("LP.lp")

        # Solve optimisation problem
        status = model.solve()

        # Show problem solution
        print(f"status: {model.status}, {pl.LpStatus[model.status]}")
        print(f"objective: {model.objective.value()}")

        # Compos results
        result = pd.DataFrame(((var.name, var.value()) for var in model.variables() if var.name.startswith("x")), columns=["Crop", "variableCostsCrops [€/Ton]"])
        result["Crop"] = result.apply(lambda x: x["Crop"].replace("x_", ""), axis=1)


        
        return result
        

In [ ]:
ccc = CostCropComputer(data)

x = ccc.main()

In [ ]:
x

In [26]:
x.to_csv("./../complete_execution_andalucia/data/use_case_greece/metadata/Crop-costs-deploy.csv", index=False)